In [ ]:
# Run NHM from pynhm to generate performance metrics

In [ ]:
#%env NUMBA_NUM_THREADS=
#%env NUMBA_THREADING_LAYER=tbb

In [ ]:
import pathlib as pl
import shutil

import pynhm
from pynhm.base.control import Control
from pynhm.base.model import Model
from pynhm.utils.parameters import PrmsParameters

output_vars = [
    'albedo',
    'gwres_flow',
    'gwres_sink',
    'hru_actet',
    'hru_impervstor',
    'hru_intcpevap',
    'hru_rain',
    'hru_snow',
    'infil',
    'pkwater_equiv',
    'potet',
    'snowcov_area',
    'soil_moist',
    'sroff',
    'ssres_flow',
    'swrad',
    'tavgc',
    'tmaxc',
    'tminc',
    'seg_outflow',
]

def time_run_nhm(domain_dir, control_file, calc_method='numba', output_dir=None):
    
    def run_nhm():
        params = PrmsParameters.load(domain_dir / 'myparam.param')
        control = Control.load(control_file, params=params)

        budget_type = None
        processes = [
            pynhm.PRMSSolarGeometry,
            pynhm.PRMSAtmosphere,
            pynhm.PRMSCanopy,
            pynhm.PRMSSnow,
            pynhm.PRMSRunoff,
            pynhm.PRMSSoilzone,
            pynhm.PRMSGroundwater,
            pynhm.PRMSChannel,
        ]

        model = Model(
            *processes,
            control=control,
            input_dir=domain_dir,
            budget_type=budget_type,
            calc_method=calc_method
            # load_n_time_batches=3,
        )

        # not sure this is necessary but to be sure
        if 'yes' in control_file.name:
            model.run(netcdf_dir=output_dir, output_vars=output_vars)
        else:
            model.run()
            
        return

    if "conus" in str(domain_dir): 
        result = %timeit -o -n1 -r1 run_nhm()
    else:
        result = %timeit -o run_nhm()
        
    return result


In [ ]:
pynhm_run_dir = pl.Path('/Users/jamesmcc/usgs/data/pynhm/performance_runs/pynhm')  # just for output
test_dom_dir = pl.Path('/Users/jamesmcc/usgs/pynhm/test_data/')
conus_dom_dir = pl.Path('/Users/jamesmcc/usgs/data/pynhm/conus_2yr')

dom_dirs = [
    # test_dom_dir / 'hru_1', 
    # test_dom_dir / 'drb_2yr', 
    # test_dom_dir / 'ucb_2yr',
    conus_dom_dir,
]

In [ ]:
# Actually passing the control file is moot since it dosent control output
# for pynhm, but... we'll use the control file name to key io in the model setup.
control_dir = pl.Path('/Users/jamesmcc/usgs/data/pynhm/performance_runs/PRMS/control_files')

run_dict = {}
for dom in dom_dirs:
    for meth in ['numpy', 'numba']:
        if dom == conus_dom_dir and meth == 'numpy':
            continue
        for io in ['yes', 'no']:
            run_dir = pynhm_run_dir / f"{dom.name}_io_{io}_method_{meth}"
            if io == 'yes':
                if run_dir.exists():
                    shutil.rmtree(run_dir)
                run_dir.mkdir(parents=True)
            src_dir = f"{dom}"

            n_hru_desc = 'multi'
            if 'hru_1' in dom.name:
                n_hru_desc = 'single'
            control_file = control_dir / f"control.{n_hru_desc}_hru_{io}_io"            
            assert control_file.exists()
            
            run_dict[f"{run_dir}"] = {
                'src': src_dir, 
                'io': io, 
                'method': meth, 
                'control_file': control_file
            }

In [ ]:
run_dict

In [ ]:
results = {}
for run_dir, run_info in run_dict.items():
    print(run_dir)
    output_dir = None
    if run_info['io'] == 'yes': 
        output_dir = pl.Path(run_dir)
        continue
    else:
        #continue
        pass
    
    results[run_dir] =  time_run_nhm(
        domain_dir=pl.Path(run_info['src']),
        control_file = run_info['control_file'],
        calc_method=run_info['method'],
        output_dir=output_dir
    )

In [ ]:
results

In [ ]:
import pickle
for path, result in results.items():
    path = pl.Path(path)
    pkl_path = path.parent.parent /  (f"results/{path.parent.name}_{path.name}.pkl")
    print(pkl_path)
    
    with open(pkl_path, "wb") as output_file:
        pickle.dump(result, output_file)

In [ ]:
results2 = {}
files = pl.Path('/Users/jamesmcc/usgs/data/pynhm/performance_runs/results/').glob('*.pkl')
for ff in files: 
    print(ff)
    with open(ff, "rb") as input_file:
        results2[ff.name[0:-4]] = pickle.load(input_file)

In [ ]:
results2